In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
import warnings
warnings.filterwarnings("ignore")

In [78]:
import src.config as config

In [79]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

2025-02-16 14:57:16,434 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-16 14:57:16,490 INFO: Initializing external client
2025-02-16 14:57:16,491 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-16 14:57:17,518 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213632


In [80]:
# create feature view (if it doesn't exist yet)
# This feature view only uses on feature group, so the query is trivial
try:
    # create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')


# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view already existed. Skip creation.


In [81]:
ts_data, _ = feature_view.training_data(
    description='Time-series hourly taxi rides',
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (63.16s) 


In [82]:
# drop `pickup_ts` column
ts_data.drop('pickup_ts', axis=1, inplace=True)

# sort by `pickup_location_id` and `pickup_hour`
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
ts_data


,pickup_hour,rides,pickup_location_id
4255016,2022-01-01 00:00:00+00:00,0,1
313,2022-01-01 01:00:00+00:00,0,1
4166856,2022-01-01 02:00:00+00:00,0,1
6641532,2022-01-01 03:00:00+00:00,0,1
6278707,2022-01-01 04:00:00+00:00,1,1
...,...,...,...
2877714,2025-02-15 03:00:00+00:00,3,265
2414701,2025-02-15 04:00:00+00:00,1,265
2108017,2025-02-15 05:00:00+00:00,2,265
3750504,2025-02-15 06:00:00+00:00,2,265


In [83]:
# from src.plot import plot_ts
from typing import Optional, List
import pandas as pd
import plotly.express as px 

def plot_ts(
    ts_data: pd.DataFrame,
    locations: Optional[List[int]] = None
    ):
    """
    Plot time-series data
    """
    ts_data_to_plot = ts_data[ts_data.pickup_location_id.isin(locations)] if locations else ts_data

    fig = px.line(
        ts_data_to_plot,
        x="pickup_hour",
        y="rides",
        color='pickup_location_id',
        template='none',
    )

    fig.show()

plot_ts(ts_data, locations=[43])

In [84]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28*1, # one month
    step_size=23,
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')
print(f'{features.shape=}')
print(f'{targets.shape=}')

100%|██████████| 265/265 [04:21<00:00,  1.01it/s]


features_and_target.shape=(294680, 675)
features.shape=(294680, 674)
targets.shape=(294680,)


In [85]:
features_and_target

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29 00:00:00+00:00,1,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2022-01-29 23:00:00+00:00,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,2.0,2.0,0.0,1.0,2.0,0.0,0.0,2022-01-30 22:00:00+00:00,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,2.0,1.0,0.0,1.0,2022-01-31 21:00:00+00:00,1,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2022-02-01 20:00:00+00:00,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294675,3.0,6.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,2.0,2.0,1.0,4.0,2.0,0.0,0.0,2025-02-11 05:00:00+00:00,265,1.0
294676,2.0,0.0,2.0,6.0,0.0,3.0,1.0,2.0,5.0,2.0,...,1.0,3.0,2.0,0.0,2.0,0.0,0.0,2025-02-12 04:00:00+00:00,265,1.0
294677,2.0,3.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,...,1.0,0.0,3.0,2.0,2.0,3.0,0.0,2025-02-13 03:00:00+00:00,265,1.0
294678,2.0,1.0,4.0,0.0,5.0,1.0,1.0,3.0,4.0,0.0,...,2.0,4.0,2.0,1.0,9.0,3.0,7.0,2025-02-14 02:00:00+00:00,265,0.0


In [86]:
features_and_target['pickup_hour'] = pd.to_datetime(features_and_target['pickup_hour'], utc=True)

In [87]:
features_and_target.sort_values('pickup_hour', inplace=True)

In [88]:
from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from src.data_split import train_test_split


features['pickup_hour'] = pd.to_datetime(features['pickup_hour'], utc=True)

# training data -> from January 2022 up until 2 months ago
# test data -> last 2 months
cutoff_date = pd.to_datetime(date.today() - timedelta(days=28*2), utc=True)
print(f'{cutoff_date=}') # Debugging


X_train, y_train, X_test, y_test = train_test_split(
features_and_target,
cutoff_date,
target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

cutoff_date=Timestamp('2024-12-22 00:00:00+0000', tz='UTC')
X_train.shape=(286995, 674)
y_train.shape=(286995,)
X_test.shape=(7685, 674)
y_test.shape=(7685,)


In [89]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
    
    # sort X_train by `pikup_hour` inplace
    # so the TimeSeriesSplit will split the data in a consistent way
    #X_train.sort_values('pickup_hour', inplace=True)
    # y_train.sort_values('pickup_hour', inplace=True)

    tss = TimeSeriesSplit(n_splits=4)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [90]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

[I 2025-02-16 15:06:09,355] A new study created in memory with name: no-name-a653fa40-8604-421c-8f63-4b847d57ce06
[I 2025-02-16 15:09:22,242] Trial 0 finished with value: 2.765235499442462 and parameters: {'num_leaves': 243, 'feature_fraction': 0.966231192756605, 'bagging_fraction': 0.6980670414989036, 'min_child_samples': 12}. Best is trial 0 with value: 2.765235499442462.
[I 2025-02-16 15:11:19,283] Trial 1 finished with value: 2.797532861001184 and parameters: {'num_leaves': 150, 'feature_fraction': 0.4093286665241997, 'bagging_fraction': 0.8421553518738716, 'min_child_samples': 16}. Best is trial 0 with value: 2.765235499442462.
[I 2025-02-16 15:12:56,386] Trial 2 finished with value: 2.7919408911400225 and parameters: {'num_leaves': 35, 'feature_fraction': 0.5422397551484447, 'bagging_fraction': 0.5306253132361298, 'min_child_samples': 13}. Best is trial 0 with value: 2.765235499442462.
[I 2025-02-16 15:14:18,933] Trial 3 finished with value: 2.765663035503055 and parameters: {'nu

In [91]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 163, 'feature_fraction': 0.6755137632255699, 'bagging_fraction': 0.7317109108834197, 'min_child_samples': 23}


In [92]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x00000275239CACA0>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.7317109108834197,
                               feature_fraction=0.6755137632255699,
                               min_child_samples=23, num_leaves=163))])

In [93]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=4.0460


In [94]:
import joblib
from src.paths import MODELS_DIR

joblib.dump(pipeline, MODELS_DIR / 'model.pkl')

['C:\\Users\\ijato\\Documents\\taxi_demand_predictor\\models\\model.pkl']

In [95]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor with a bit of hyper-parameter tuning",
    input_example=X_train.sample(),
    model_schema=model_schema
)
#model.save(str(MODELS_DIR/'model.pkl'))
# model.save(MODELS_DIR / 'model.pkl')
model.save(str(MODELS_DIR/'model.pkl'))

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/1464123 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/3399 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/60849 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1213632/models/taxi_demand_predictor_next_hour/2


Model(name: 'taxi_demand_predictor_next_hour', version: 2)